In [ ]:
import pandas as pd
import time

from multisourcetaskgenerator import *
from blocking import *
from feature_vector_creation import *
from setting_utils import * 

# Starting point for ALMSERgen. 
1. Define configuration
    - sources: amount of sources to be generated
    - id_attr: idnetifying attributes of the domain
    - main_path : path to output the generated tasks
    - vpo_values : amount of groups of data sources with the same value pattern. Dont't forget to normalize it before analysis!!!
    - eo_values : entity overlap levels
    - vh_values : value heterogeneity levels
    
2. ALMSERgen will generate the tasks (sources and feature vectors)
3. Along with the tasks the foloowing files are generated.
    - train/test split using the cc of the complete graph
    - unsupervised results
    - passive learning results
    - distribution of connected components sizes (bar chart)
    - naive transfer learning heatmap as csv
    - profiling information on the task
    

In [ ]:
sources=6
id_attr= ['title','release','artist','country']
original_ds = pd.read_csv("msd_1K.csv")
main_path="continuum_tasks"

#define the values of the dimensions
vpo_values = [2] #2 groups
eo_values = [1.0]
vh_values = [0.2]


for s_v in eo_values:
    for g_v in vpo_values:
        for cc_v in vh_values:
            config = dict()
            config['structure'] = s_v
            config['groups'] = g_v
            config['corner_cases'] = cc_v
            
            setting_path=main_path+"/EO%.1fVH%.1fVPO%.1f/" %(config['structure'], config['corner_cases'], config['groups']/sources,)

            if not os.path.exists(setting_path):
                os.makedirs(setting_path)

            start = time.time()

            setting = MultiSourceTaskGenerator(original_ds, sources, 'id', id_attr,
                                  config)
            setting.generate()


            setting.write_sources(setting_path)

            #blocking : will create the path blocked_pairs
            get_pairs_for_fv(setting_path, 'id')

            #feature vector : will create the path feature_vector_files
            createVectorsFromFiles(setting_path, 'id')

            #write train and test data
            write_and_profile_setting(setting_path)
            end = time.time()
            print("Creating one multi-source setting in : %f sec." %(end - start))

                
